# Deploying FastKafka using Docker

## Building docker image

To build a docker image for a FastKafka project we need the following things,

1. A library which is built using FastKafka.
2. A file in which requirements are specified. It could be a requirements.txt file or a setup.py or it could even be a wheel file.
3. A Dockerfile to build an image which will include the above two files

### Creating FastKafka code

Let's create a `FastKafka` based application and write to the `application.py` file based on the [tutorial](/#tutorial)

```python
# content of the "application.py" file

from pydantic import BaseModel, NonNegativeFloat, Field

class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPrediction(BaseModel):
    species: str = Field(..., example="setosa", description="Predicted species")

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Iris predictions",
    kafka_brokers=kafka_brokers,
    bootstrap_servers="localhost:9092",
)

iris_species = ["setosa", "versicolor", "virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPrediction:
    prediction = IrisPrediction(species=iris_species[species_class])
    return prediction

```

### Creating requirements.txt file

The above code only requires fastkafka. So, we will add only fastkafka to the `requirements.txt` file but you can add additional requirements to it too.

```txt
fastkafka>=0.3.0
```

Here we are using `requirements.txt` to store the project's dependencies. But you can use other methods like `setup.py`, `pipenv` or even `wheel` files.

### Creating Dockerfile

```{ .dockerfile .annotate }
# (1)
FROM python:3.9-slim-bullseye
# (2)
WORKDIR /project
# (3)
COPY application.py requirements.txt /project/
# (4)
RUN pip install --no-cache-dir --upgrade -r /project/requirements.txt
# (5)
CMD ["fastkafka", "run", "--num-workers", "2", "--kafka-broker", "production", "application:kafka_app"]
```

1. Start from the official Python base image.

2. Set the current working directory to `/project`.

    This is where we'll put the `requirements.txt` file and the `application.py` file.

3. Copy the `application.py` file and `requirementx.txt` file inside the `/project` directory.

4. Install the package dependencies in the requirements file.

    The `--no-cache-dir` option tells `pip` to not save the downloaded packages locally, as that is only if `pip` was going to be run again to install the same packages, but that's not the case when working with containers.

    The `--upgrade` option tells `pip` to upgrade the packages if they are already installed.

5. Set the **command** to run the `fastkafka run` command.

    `CMD` takes a list of strings, each of these strings is what you would type in the command line separated by spaces.

    This command will be run from the **current working directory**, the same `/project` directory you set above with `WORKDIR /project`.

    We supply additional parameters `--num-workers` and `--kafka-broker` for the run command. Finally, we specify the location of our `fastkafka` app location as a command argument.
    
    To learn more about `fastkafka run` command please check the [cli docs](/cli/fastkafka/#fastkafka-run).


### Build the Docker Image

Now that all the files are in place, let's build the container image.

* Go to the project directory (where your `Dockerfile` is, containing your `application.py` file).
* Build your FastKafka image:
```cmd
docker build -t fastkafka_project_image .
```

This will build a docker image called `fastkafka_project_image` with `latest` tag

### Start the Docker Container

Run a container based on the built image:
```cmd
docker run -d --name fastkafka_project_container fastkafka_project_image
```

## Additional Security

We recommend using [`trivy`](https://github.com/aquasecurity/trivy) to scan the built image for any vulnerabilities and to fix it based on its recommendations.

## Example repo

A `fastkafka` based library which uses above mentioned Dockerfile to build a docker image can be found [here](https://github.com/airtai/sample_fastkafka_project/)